In [7]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import gzip 
import json 
import polars as pl
import re
import ast
from sklearn.preprocessing import LabelEncoder

In [2]:
steam_reviews_parquet = "./data/reviews.parquet"

products_parquet = "./data/products_X.parquet"

sorted_reviews_parquet = "./data/sorted_reviews.parquet"

# Preprocess data

In [3]:
review_df = pd.read_parquet(sorted_reviews_parquet)

In [4]:
print(review_df)

        hours  products  product_id  \
0         7.5     925.0       35130   
1         0.0     925.0          70   
2         NaN     925.0          40   
3         NaN     925.0          30   
4         4.9     925.0       33120   
...       ...       ...         ...   
899832    2.0       5.0      349040   
899833    0.2       5.0      238460   
899834    0.5       5.0      204360   
899835    0.3       5.0      674940   
899836    0.1       5.0         440   

                                                     text  found_funny  \
0       良作アクションなので懐事情に合わせて買うもよし、セールを待つもよし\nただしキーボードでの操...          NaN   
1       紛れもないFPS界の金字塔。マスト！\nまあ今の人はSource版やるとよいです。\n今は無...          NaN   
2       マップとかまんまQuakeのDMなので懐かしさいっぱい\nFPS界の先輩に対するリスペクトっ...          NaN   
3       製品とかになる前にANC鯖とか中野鯖などで死ぬほどやった\nCSのストイックさというか死んだ...          NaN   
4       Alien Shooter: Vengeanceを持ってたらいつの間にか2が増えていた\nな...          NaN   
...                                                   ...          ...   
899832       

In [5]:
relevant_df = review_df[['user_id', 'product_id', 'review_date']]
print(relevant_df)

                  user_id  product_id review_date
0       76561197960267615       35130  2010-12-10
1       76561197960267615          70  2010-12-10
2       76561197960267615          40  2010-12-10
3       76561197960267615          30  2010-12-10
4       76561197960267615       33120  2010-12-10
...                   ...         ...         ...
899832  76561198801034135      349040  2018-01-03
899833  76561198801034135      238460  2018-01-04
899834  76561198801034135      204360  2018-01-04
899835  76561198801034135      674940  2018-01-04
899836  76561198801034135         440  2018-01-05

[899837 rows x 3 columns]


In [6]:
# we assume dataset is already sorted by date
# to improve performance, try to filter out users with less than 5 games
user_counts = relevant_df['user_id'].value_counts()
good_users = user_counts[user_counts >= 5].index
df = relevant_df[relevant_df['user_id'].isin(good_users)]

In [8]:
# convert product_id to sequential ints
encoder = LabelEncoder()
df['product_id'] = encoder.fit_transform(df['product_id'])

In [10]:
# create list of interactions(product_ids) per user
#################### set max sequence length of interactions to fit with model config ##########################
MAX_SEQ_LENGTH = 50

interactions = df.groupby('user_id')['product_id'].apply(list).reset_index()
interactions['product_id'] = interactions['product_id'].apply(lambda x: x[-MAX_SEQ_LENGTH:])
print(interactions)

                 user_id                                         product_id
0      76561197960267615  [694, 6, 3, 2, 649, 568, 863, 1067, 576, 1091,...
1      76561197960269475                   [432, 48, 923, 1349, 1203, 9747]
2      76561197960270993          [891, 1242, 3491, 1768, 1225, 3233, 4556]
3      76561197960271248     [1537, 6031, 4881, 271, 5064, 700, 3917, 9484]
4      76561197960271541                        [565, 259, 1354, 1366, 283]
...                  ...                                                ...
92107  76561198729247143                 [3521, 2916, 1675, 134, 2753, 161]
92108  76561198736194466                  [7232, 19, 3713, 545, 2891, 2543]
92109  76561198744164375                  [6991, 1558, 134, 1537, 3386, 24]
92110  76561198797433502                    [4992, 6946, 10484, 3357, 3386]
92111  76561198801034135                [3841, 4660, 1810, 1249, 10201, 19]

[92112 rows x 2 columns]


In [107]:
# create csv file with columns user_id, item_id, timestamp according to recbole format
processed_data = []
for _, row in interactions.iterrows():
    user_id = row['user_id']
    sequence = row['product_id'][-MAX_SEQ_LENGTH:]
    for i, item_id in enumerate(sequence):
        # we don't need exact times, just relative ordering!
        timestamp = i + 1
        processed_data.append([user_id, item_id, float(timestamp)])
        
interaction_df = pd.DataFrame(processed_data, columns=['user_id:token', 'item_id:token', 'timestamp:float'])
interaction_df.to_csv('./data/interactions/interactions.inter', index=False)

# Set BERT4Rec Configurations

In [14]:
import yaml
from recbole.quick_start import run_recbole, load_data_and_model

In [120]:
recbole_config = {
    # General Configs
    #"MODEL_TYPE": "sequential",
    "MODEL": "BERT4Rec",
    #"DATA_PATH": "./dataset/", 
    "dataset": "interactions",
    "data_path": ".\\data\\", # windows specifically; should use "/" to be more general but debugging rn
    #"data_path": "C:\\Users\\ruiqi\\OneDrive - UC San Diego\\Desktop\\CS_HW\\CSE258\\data\\",

    # Dataset Configs
    "field_separator": ",",
    "USER_ID_FIELD": "user_id",
    "ITEM_ID_FIELD": "item_id",
    "TIME_FIELD": "timestamp",
    "load_col": {
        "inter": ['user_id', 'item_id', 'timestamp']
    },
    "MAX_SEQ_LENGTH": 50,  # Should match dataset max_seq_length

    # Training Configs
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "learning_rate": 0.001,
    "num_epochs": 10,
    "optimizer": "adam",
    "train_neg_sample_args": None,  # Disable negative sampling
    "loss_type": "CE",               # Cross-Entropy loss

    # Model Specific Configs
    "hidden_size": 128,
    "num_layers": 2,
    "num_heads": 4,
    "dropout_prob": 0.1,
    "initializer_range": 0.02,

    # Evaluation Configs
    "metrics": ["Recall", "NDCG", "Hit"],
    "topk": [10, 25, 50],
    "valid_metric": "NDCG@10",
    "eval_setting": "TO 1:1",
}

In [121]:
config_file_path = "bert4rec.yaml"
with open(config_file_path, "w") as file:
    yaml.dump(recbole_config, file, default_flow_style=False)

# Run Recbole BERT4Rec Model

In [ ]:
run_recbole(model='BERT4Rec', config_file_list=['bert4rec.yaml'])

02 Dec 00:51    INFO  ['C:\\Users\\ruiqi\\anaconda3\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\ruiqi\\AppData\\Roaming\\jupyter\\runtime\\kernel-5ec93ca9-0a18-4bb0-93c0-1f5a339637b8.json']
02 Dec 00:51    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = .\data\interactions
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 64
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'ful

Train     0:   0%|                                               | 1/9243 [00:01<3:50:28,  1.50s/it]:   0%|                                               | 2/9243 [00:02<3:14:30,  1.26s/it]:   0%|                                               | 3/9243 [00:02<2:46:09,  1.08s/it]:   0%|                                               | 4/9243 [00:03<2:27:08,  1.05it/s]:   0%|                                               | 5/9243 [00:04<2:14:51,  1.14it/s]:   0%|                                               | 6/9243 [00:04<2:05:33,  1.23it/s]:   0%|                                               | 7/9243 [00:05<1:59:48,  1.28it/s]:   0%|                                               | 8/9243 [00:06<1:55:53,  1.33it/s]:   0%|                                               | 9/9243 [00:06<1:52:49,  1.36it/s]:   0%|                                              | 10/9243 [00:07<1:49:04,  1.41it/s]:   0%|                                              | 11/9243 [00:08<1:47:46,  1.43it/s]:   0%|   

Train     0:   1%|▍                                             | 93/9243 [01:09<1:56:14,  1.31it/s]:   1%|▍                                             | 94/9243 [01:10<1:56:04,  1.31it/s]:   1%|▍                                             | 95/9243 [01:10<1:55:00,  1.33it/s]:   1%|▍                                             | 96/9243 [01:11<1:54:35,  1.33it/s]:   1%|▍                                             | 97/9243 [01:12<1:54:54,  1.33it/s]:   1%|▍                                             | 98/9243 [01:13<1:55:50,  1.32it/s]:   1%|▍                                             | 99/9243 [01:13<1:55:36,  1.32it/s]:   1%|▍                                            | 100/9243 [01:14<1:56:10,  1.31it/s]:   1%|▍                                            | 101/9243 [01:15<1:55:43,  1.32it/s]:   1%|▍                                            | 102/9243 [01:16<1:55:01,  1.32it/s]:   1%|▌                                            | 103/9243 [01:16<1:54:01,  1.34it/s]:   1%|▌  

Train     0:   2%|▉                                            | 185/9243 [02:20<1:50:51,  1.36it/s]:   2%|▉                                            | 186/9243 [02:21<1:50:35,  1.36it/s]:   2%|▉                                            | 187/9243 [02:22<1:50:20,  1.37it/s]:   2%|▉                                            | 188/9243 [02:23<1:49:10,  1.38it/s]:   2%|▉                                            | 189/9243 [02:23<1:49:01,  1.38it/s]:   2%|▉                                            | 190/9243 [02:24<1:48:14,  1.39it/s]:   2%|▉                                            | 191/9243 [02:25<1:50:46,  1.36it/s]:   2%|▉                                            | 192/9243 [02:26<1:49:45,  1.37it/s]:   2%|▉                                            | 193/9243 [02:26<1:49:42,  1.37it/s]:   2%|▉                                            | 194/9243 [02:27<1:49:13,  1.38it/s]:   2%|▉                                            | 195/9243 [02:28<1:49:11,  1.38it/s]:   2%|▉  

Train     0:   3%|█▎                                           | 277/9243 [03:30<1:47:25,  1.39it/s]:   3%|█▎                                           | 278/9243 [03:31<1:46:19,  1.41it/s]:   3%|█▎                                           | 279/9243 [03:31<1:47:08,  1.39it/s]:   3%|█▎                                           | 280/9243 [03:32<1:48:01,  1.38it/s]:   3%|█▎                                           | 281/9243 [03:33<1:57:13,  1.27it/s]:   3%|█▎                                           | 282/9243 [03:34<2:05:05,  1.19it/s]:   3%|█▍                                           | 283/9243 [03:35<2:30:29,  1.01s/it]:   3%|█▍                                           | 284/9243 [03:37<2:41:16,  1.08s/it]:   3%|█▍                                           | 285/9243 [03:38<2:48:28,  1.13s/it]:   3%|█▍                                           | 286/9243 [03:39<2:50:30,  1.14s/it]:   3%|█▍                                           | 287/9243 [03:40<2:43:14,  1.09s/it]:   3%|█▍ 

...

Train     1:   2%|▉                                            | 184/9243 [02:45<2:03:57,  1.22it/s]:   2%|▉                                            | 185/9243 [02:46<2:01:30,  1.24it/s]:   2%|▉                                            | 186/9243 [02:47<1:59:51,  1.26it/s]:   2%|▉                                            | 187/9243 [02:48<1:58:16,  1.28it/s]:   2%|▉                                            | 188/9243 [02:48<1:58:54,  1.27it/s]:   2%|▉                                            | 189/9243 [02:49<1:59:28,  1.26it/s]:   2%|▉                                            | 190/9243 [02:50<2:00:08,  1.26it/s]:   2%|▉                                            | 191/9243 [02:51<2:00:40,  1.25it/s]:   2%|▉                                            | 192/9243 [02:52<1:59:51,  1.26it/s]:   2%|▉                                            | 193/9243 [02:52<1:59:59,  1.26it/s]:   2%|▉                                            | 194/9243 [02:53<1:59:05,  1.27it/s]:   2%|▉  

Train     1:   3%|█▎                                           | 276/9243 [04:06<2:18:02,  1.08it/s]:   3%|█▎                                           | 277/9243 [04:07<2:17:11,  1.09it/s]:   3%|█▎                                           | 278/9243 [04:08<2:16:52,  1.09it/s]:   3%|█▎                                           | 279/9243 [04:09<2:16:07,  1.10it/s]:   3%|█▎                                           | 280/9243 [04:10<2:16:15,  1.10it/s]:   3%|█▎                                           | 281/9243 [04:11<2:16:05,  1.10it/s]:   3%|█▎                                           | 282/9243 [04:12<2:15:33,  1.10it/s]:   3%|█▍                                           | 283/9243 [04:12<2:14:29,  1.11it/s]:   3%|█▍                                           | 284/9243 [04:13<2:14:32,  1.11it/s]:   3%|█▍                                           | 285/9243 [04:14<2:15:29,  1.10it/s]:   3%|█▍                                           | 286/9243 [04:15<2:15:19,  1.10it/s]:   3%|█▍ 

Train     1:   4%|█▊                                           | 368/9243 [05:28<2:47:27,  1.13s/it]:   4%|█▊                                           | 369/9243 [05:29<2:41:38,  1.09s/it]:   4%|█▊                                           | 370/9243 [05:30<2:43:49,  1.11s/it]:   4%|█▊                                           | 371/9243 [05:32<2:55:18,  1.19s/it]:   4%|█▊                                           | 372/9243 [05:33<2:51:03,  1.16s/it]:   4%|█▊                                           | 373/9243 [05:34<2:47:49,  1.14s/it]:   4%|█▊                                           | 374/9243 [05:35<2:48:48,  1.14s/it]:   4%|█▊                                           | 375/9243 [05:36<2:45:36,  1.12s/it]:   4%|█▊                                           | 376/9243 [05:37<2:41:14,  1.09s/it]:   4%|█▊                                           | 377/9243 [05:38<2:41:44,  1.09s/it]:   4%|█▊                                           | 378/9243 [05:39<2:41:11,  1.09s/it]:   4%|█▊ 

Train     1:   5%|██▏                                          | 460/9243 [06:59<2:01:08,  1.21it/s]:   5%|██▏                                          | 461/9243 [07:00<2:00:41,  1.21it/s]:   5%|██▏                                          | 462/9243 [07:01<2:00:57,  1.21it/s]:   5%|██▎                                          | 463/9243 [07:01<2:01:27,  1.20it/s]:   5%|██▎                                          | 464/9243 [07:02<1:59:58,  1.22it/s]:   5%|██▎                                          | 465/9243 [07:03<1:59:42,  1.22it/s]:   5%|██▎                                          | 466/9243 [07:04<1:58:59,  1.23it/s]:   5%|██▎                                          | 467/9243 [07:05<2:00:29,  1.21it/s]:   5%|██▎                                          | 468/9243 [07:05<1:59:21,  1.23it/s]:   5%|██▎                                          | 469/9243 [07:06<1:59:08,  1.23it/s]:   5%|██▎                                          | 470/9243 [07:07<2:00:29,  1.21it/s]:   5%|██▎

Train     1:   6%|██▋                                          | 552/9243 [08:35<2:58:12,  1.23s/it]:   6%|██▋                                          | 553/9243 [08:36<2:42:27,  1.12s/it]:   6%|██▋                                          | 554/9243 [08:37<2:49:46,  1.17s/it]:   6%|██▋                                          | 555/9243 [08:38<2:39:11,  1.10s/it]:   6%|██▋                                          | 556/9243 [08:39<2:26:30,  1.01s/it]:   6%|██▋                                          | 557/9243 [08:39<2:20:01,  1.03it/s]:   6%|██▋                                          | 558/9243 [08:40<2:13:38,  1.08it/s]:   6%|██▋                                          | 559/9243 [08:41<2:12:09,  1.10it/s]:   6%|██▋                                          | 560/9243 [08:42<2:10:58,  1.10it/s]:   6%|██▋                                          | 561/9243 [08:43<2:09:44,  1.12it/s]:   6%|██▋                                          | 562/9243 [08:44<2:12:03,  1.10it/s]:   6%|██▋

In [ ]:
# debug
from recbole.config import Config
from recbole.data import create_dataset
from recbole.utils import init_seed

In [109]:
config = Config(model='BERT4Rec', config_file_list=['bert4rec.yaml'])

try:
    dataset = create_dataset(config)
    print("Dataset loaded successfully!")
    print("Dataset Fields:", dataset.field2type)
except Exception as e:
    print(f"Error: {e}")

Dataset loaded successfully!
Dataset Fields: {'user_id': <FeatureType.TOKEN: 'token'>, 'item_id': <FeatureType.TOKEN: 'token'>}


In [110]:
print("Dataset Fields:", dataset.field2type)
print("Interaction Fields:", dataset.inter_feat)

Dataset Fields: {'user_id': <FeatureType.TOKEN: 'token'>, 'item_id': <FeatureType.TOKEN: 'token'>}
Interaction Fields:         user_id  item_id
0             1        1
1             1        2
2             1        3
3             1        4
4             1        5
...         ...      ...
867850    92112     5785
867851    92112     2571
867852    92112     1588
867853    92112      791
867854    92112     1147

[867855 rows x 2 columns]
